### Homework 3: Financial Ratio Quantile Strategies - Hengxu Zhu 12368409

<font color='red'>Note: I have asked professor to extend my deadline for this homework as I went to another city for interview. I have also double checked with TA. Please let me know if you did not receive my email. Thanks!</font>

## Import Packages

In [1]:
%matplotlib inline 
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
import datetime
from statsmodels.tsa.ar_model import AutoReg 
import warnings
warnings.filterwarnings('ignore')

## Load Data

In [2]:
quandl.ApiConfig.api_key = '_MNv7zC_784tGzbfxhxP' 

#### Zacks Tables

For the 6 tables Zacks provide, they are Fundamentals Condensed(FC), Fundamentals Ratios(FR), Master Table(MT), Market Value Supplement(MKTV), Shares Out Supplement(SHRS) and Historical Daily Maintenance(HDM). 

#### Calculate Financial Ratios 

I changed this part of the start date as the following calculation using fillna(method='ffill') deleted many data. Using a start date of 2014-07-31 can ensure that all ratios available starting 2015. 

In [3]:
start = '2014-07-31' 
end = '2022-01-31'

From the homework instruction, we know this process should be done after we define the universe. However, since our universe requires that all financial ratios used in the strategy to be feasible in calculation, the functions have to be extracted first so that we can use them to filter the tickers in our universe. Therefore, let's define some functions to calculate these ratios first. 

Before defining the functions, we need to select all the coloumns needed from the Zacks Tables. After reading the homework instruction and Ed discussion, I selected columns accordingly and did the following formulation on these selected data. 

Note: 
1. Table MT and HDM are not used as they does not contain any useful information as explained from homework instruction and Nasdaq documents. MT are just descriptions for the tickers and HDM are the corporate actions. Here is an example of the MT data for Eli Lilly. 

In [4]:
quandl.get_table('ZACKS/MT', ticker = 'LLY')

,m_ticker,ticker,comp_name,comp_name_2,exchange,currency_code,ticker_type,active_ticker_flag,comp_url,sic_4_code,...,per_end_month_nbr,mr_split_date,mr_split_factor,comp_cik,country_code,country_name,comp_type,optionable_flag,sp500_member_flag,asset_type
None,,,,,,,,,,,,,,,,,,,,,
0,LLY,LLY,Eli Lilly,Eli Lilly and Company,NYSE,USD,S,Y,http://www.lilly.com,2834,...,12,1997-10-16,2.0,0000059478,US,UNITED STATES,1,Y,Y,COM


In [5]:
def zacks(ticker, start, end): 
    
    # Download raw data from Quandl
    fc = quandl.get_table('ZACKS/FC', ticker = ticker, per_end_date = {'gte':start,'lte':end}) 
    fc = fc[fc['per_type']=='Q']
    fr = quandl.get_table('ZACKS/FR', ticker = ticker, per_end_date = {'gte':start,'lte':end}) 
    fr = fr[fr['per_type']=='Q']
    mktv = quandl.get_table('ZACKS/MKTV', ticker = ticker, per_end_date = {'gte':start,'lte':end}) 
    shrs = quandl.get_table('ZACKS/SHRS', ticker = ticker, per_end_date = {'gte':start,'lte':end}) 

    # Select columns
    fc = fc[['per_end_date', 'filing_date', 'eps_diluted_net', 'basic_net_eps', 'net_lterm_debt', 'tot_lterm_debt']].rename(columns={'per_end_date': 'date'})
    fr = fr[['per_end_date', 'ret_invst', 'tot_debt_tot_equity']].rename(columns={'per_end_date': 'date'}) 
    mktv = mktv[['per_end_date', 'mkt_val']].rename(columns={'per_end_date': 'date'})
    shrs = shrs[['per_end_date', 'shares_out']].rename(columns={'per_end_date': 'date'})
    
    # Sort and merge the data into one table 
    fc = fc.sort_values(by=['date']).set_index('date')
    fr = fr.sort_values(by=['date']).set_index('date')
    mktv = mktv.sort_values(by=['date']).set_index('date')
    shrs = shrs.sort_values(by=['date']).set_index('date')
    df = fc.merge(fr, left_on='date', right_on='date') 
    df = df.merge(mktv, left_on='date', right_on='date') 
    df = df.merge(shrs, left_on='date', right_on='date') 
    
    return df

In [6]:
def adj_close(ticker, start, end): 
    adj_close = quandl.get_table('QUOTEMEDIA/PRICES', ticker=ticker,date = {'gte':start,'lte':end}) 
    adj_close = adj_close[['date','adj_close']] 
    adj_close = adj_close.sort_values(by=['date']).set_index('date') 
    return adj_close 

1. Debt to Market Cap

In [7]:
def debt_to_market_cap(adj_close, zacks): 
    zacks = zacks[['filing_date', 'tot_debt_tot_equity']]
    df = zacks.merge(adj_close, left_on='date', right_on='date') 
    df = df.rename(columns={'filing_date': 'date'}).set_index('date')
    df['filing_adj_close'] = df['adj_close'] 
    df = df.drop(columns='adj_close') 
    df = adj_close.join(df) 
    df['filing_adj_close'] =  df['filing_adj_close'].shift().fillna(method='ffill')
    df['tot_debt_tot_equity'] =  df['tot_debt_tot_equity'].shift().fillna(method='ffill') 
    df['adj_close_ratio'] = df['filing_adj_close']/df['adj_close'] 
    df['debt_to_market_cap'] = df['tot_debt_tot_equity']*df['adj_close_ratio']
    df = df[['debt_to_market_cap']].dropna()
    return df 

2. Return on Investment 

In [8]:
def return_on_investment(adj_close, zacks): 
    zacks = zacks[['filing_date', 'ret_invst', 'mkt_val', 'net_lterm_debt', 'tot_lterm_debt']]
    df = zacks.merge(adj_close, left_on='date', right_on='date') 
    df = df.rename(columns={'filing_date': 'date'}).set_index('date')
    df['filing_adj_close'] = df['adj_close'] 
    df = df.drop(columns='adj_close') 
    df['net_lterm_debt'] = np.where(df['net_lterm_debt']=='NaN', df['tot_lterm_debt'], df['net_lterm_debt']) 
    df['ret_r'] = df['ret_invst']*(df['mkt_val']+df['net_lterm_debt'])
    df = adj_close.join(df) 
    df['filing_adj_close'] =  df['filing_adj_close'].shift().fillna(method='ffill')
    df['mkt_val'] =  df['mkt_val'].shift().fillna(method='ffill') 
    df['net_lterm_debt'] =  df['net_lterm_debt'].shift().fillna(method='ffill')
    df['ret_r'] =  df['ret_r'].shift().fillna(method='ffill') 
    df['adj_mkt_val'] = df['mkt_val']/df['filing_adj_close']*df['adj_close'] 
    df['return_on_investment'] = df['ret_r']/(df['adj_mkt_val']+df['net_lterm_debt'])
    df = df[['return_on_investment']].dropna()
    return df

3. Price to Earnings

In [9]:
def price_to_earnings(adj_close, zacks): 
    zacks = zacks[['filing_date', 'shares_out', 'mkt_val', 'eps_diluted_net', 'basic_net_eps']]
    df = zacks.merge(adj_close, left_on='date', right_on='date') 
    df = df.rename(columns={'filing_date': 'date'}).set_index('date')
    df['filing_adj_close'] = df['adj_close'] 
    df = df.drop(columns='adj_close') 
    df['eps_diluted_net'] = np.where(df['eps_diluted_net']=='NaN', df['basic_net_eps'], df['eps_diluted_net']) 
    df['eps_diluted_net'] = np.where(df['eps_diluted_net']<0, 0.001, df['eps_diluted_net']) 
    df = adj_close.join(df) 
    df['eps_diluted_net'] =  df['eps_diluted_net'].shift().fillna(method='ffill') 
    df['price_to_earnings'] = df['adj_close']/df['eps_diluted_net']
    df = df[['price_to_earnings']].dropna()
    return df

Reconciliation

In [10]:
lly_z = zacks('LLY', start, end)
lly_c = adj_close('LLY', start, end) 
lly_dmc = debt_to_market_cap(lly_c, lly_z) 
lly_roi = return_on_investment(lly_c, lly_z) 
lly_pe = price_to_earnings(lly_c, lly_z) 
lly = lly_dmc.join(lly_roi) 
lly = lly.join(lly_pe) 
lly.loc['2020-10-28']

debt_to_market_cap       4.755473
return_on_investment     9.075546
price_to_earnings       81.908198
Name: 2020-10-28 00:00:00, dtype: float64

#### Define the Universe and Filter Tickers 

In [11]:
all_ticker = pd.read_csv('zacks-tickers.csv')
all_ticker = list(all_ticker['3262Q']) 
len(all_ticker)

8912

After creating the functions to calculate the financial ratios, four steps are taken to select the most relevant tickers. 
1. Check if adjusted close prices are available for the entire period. Since we have a reference ticker LLY, I assumed that the number of data points should be the same as this length to be available for the entire period. 
2. Check if the debt to market cap ratio is greater than 0.1 in this period. This part can be checked with step 4 as step 4 requires all calculations to be feasible. 
3. Exclude all Automotive, Financial and Insurance sector. From Zacks Sector Industry Mapping Scheme, Automotives have Sector code 5 and Finance & Insurance have code 13. Therefore, we filter the tickers by excluding those have zacks_sector_code of 5 or 13. 
4. Check if all ratios calculation are feasible, we just need to check for one day that our functions in calculating ratios can work and then check for step 2 that the debt to market cap ratio is greater than 0.1. 

The filtered 200 tickers are as following. 

In [12]:
ticker = pd.read_csv('filtered-tickers.csv')
ticker = list(ticker['ticker'])
ticker[:3]

['AAWW', 'ABT', 'ACCO']

#### Extract Financial Ratios From the Universe

1. Debt to Market Ratio

In [13]:
dmc = pd.DataFrame()
for i in range(len(ticker)): 
    c = adj_close(ticker[i], start, end) 
    z = zacks(ticker[i], start, end) 
    new = debt_to_market_cap(c, z)
    if i == 0: 
        dmc = new.rename(columns={'debt_to_market_cap': ticker[i]})
    else: 
        dmc = dmc.join(new) 
        dmc = dmc.rename(columns={'debt_to_market_cap': ticker[i]}) 
dmc = dmc.fillna(method='ffill').dropna() 
dmc.head(3)

,AAWW,ABT,ACCO,ACHC,ACIW,ACM,AIMC,AIN,AL,ALB,...,UTL,VAC,VHI,VLO,VMI,WM,WMB,XRX,YORW,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2014-12-16,1.077761,0.344122,0.991603,0.951639,1.785960,0.522817,0.996371,0.516097,2.527928,0.707040,...,1.096934,0.430473,1.177622,0.296562,0.624996,1.714133,1.322710,0.619283,0.780331,0.236887
2014-12-17,1.081837,0.337435,0.961317,0.918123,1.750644,0.513456,0.976008,0.505457,2.415796,0.679476,...,1.080416,0.421958,1.148390,0.291845,0.622506,1.708598,1.268443,0.614222,0.752233,0.230262
2014-12-18,1.051428,0.326303,0.958065,0.891804,1.692405,0.502215,0.949123,0.495659,2.344348,0.660537,...,1.068203,0.413659,1.160739,0.296310,0.624247,1.664590,1.235316,0.595207,0.732534,0.224212


2. Return on Investment

In [19]:
roi = pd.DataFrame()
for i in range(len(ticker)): 
    c = adj_close(ticker[i], start, end) 
    z = zacks(ticker[i], start, end) 
    new = return_on_investment(c, z)
    if i == 0: 
        roi = new.rename(columns={'return_on_investment': ticker[i]})
    else: 
        roi = roi.join(new) 
        roi = roi.rename(columns={'return_on_investment': ticker[i]}) 
roi = roi.fillna(method='ffill')
roi.fillna(method='bfill').head(3)

,AAWW,ABT,ACCO,ACHC,ACIW,ACM,AIMC,AIN,AL,ALB,...,UTL,VAC,VHI,VLO,VMI,WM,WMB,XRX,YORW,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2014-11-07,0.724012,2.400808,1.689198,1.101066,1.151403,2.100871,1.270410,1.287554,0.598215,3.729880,...,0.232568,1.358764,2.241500,3.606278,1.160115,1.839018,4.204579,1.394682,1.493671,2.016774
2014-11-10,0.723213,2.400808,1.630482,1.086855,1.150180,2.100871,1.258121,1.274506,0.596497,3.713401,...,0.232375,1.352474,2.241500,3.615616,1.161821,1.837530,4.250190,1.403214,1.482144,1.976047
2014-11-11,0.725079,2.400808,1.673131,1.083835,1.158175,2.100871,1.264236,1.275186,0.598584,3.677892,...,0.230014,1.335324,2.227396,3.575029,1.160425,1.842375,4.222162,1.387301,1.486192,1.964484


3. Price to Earnings

In [15]:
pe = pd.DataFrame()
for i in range(len(ticker)): 
    c = adj_close(ticker[i], start, end) 
    z = zacks(ticker[i], start, end) 
    new = price_to_earnings(c, z)
    if i == 0: 
        pe = new.rename(columns={'price_to_earnings': ticker[i]})
    else: 
        pe = pe.join(new) 
        pe = pe.rename(columns={'price_to_earnings': ticker[i]}) 
pe = pe.fillna(method='ffill').dropna() 
pe.head(3)

,AAWW,ABT,ACCO,ACHC,ACIW,ACM,AIMC,AIN,AL,ALB,...,UTL,VAC,VHI,VLO,VMI,WM,WMB,XRX,YORW,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2014-12-16,41.018182,103.211657,25.226501,137.604651,136.321429,44.098460,96.491522,84.259950,49.351393,52.588120,...,249.797755,83.376659,56.183247,16.746351,123.977606,72.286978,11.453510,30.575808,78.278454,100.921546
2014-12-17,40.863636,105.256865,26.021268,142.627907,139.071429,44.902499,98.504706,86.033588,51.642096,54.721493,...,253.616860,85.059228,57.613366,17.016970,124.473517,72.521153,11.943515,30.827744,81.202330,103.825160
2014-12-18,42.045455,108.847869,26.109576,146.837209,143.857143,45.907548,101.294907,87.734337,53.215980,56.290431,...,256.516551,86.765663,57.000458,16.760594,124.126379,74.438463,12.263797,31.812583,83.385984,106.626736


4. Adjusted Prices

In [31]:
price = pd.DataFrame()
for i in range(len(ticker)): 
    new = adj_close(ticker[i], start, end) 
    if i == 0: 
        price = new.rename(columns={'adj_close': ticker[i]})
    else: 
        price = price.join(new) 
        price = price.rename(columns={'adj_close': ticker[i]}) 
price.head(3)

,AAWW,ABT,ACCO,ACHC,ACIW,ACM,AIMC,AIN,AL,ALB,...,UTL,VAC,VHI,VLO,VMI,WM,WMB,XRX,YORW,ZBH
date,,,,,,,,,,,,,,,,,,,,,
2014-07-31,34.22,35.870372,5.651089,47.66,18.74,33.596470,27.455589,32.081296,31.075604,54.085112,...,24.393173,51.314597,64.176507,35.793926,132.511259,37.518125,33.650216,26.599469,16.085376,93.561933
2014-08-01,32.03,35.793726,5.625480,47.16,18.76,33.735013,27.981055,31.866465,31.147768,54.446620,...,24.454156,51.270014,63.688843,35.540268,132.038102,37.116951,33.008467,26.158150,16.076923,93.131849
2014-08-04,32.25,36.100311,5.565726,47.89,18.46,34.071472,28.226273,32.278223,32.067858,54.640600,...,24.339813,51.501844,65.249366,35.526176,131.837920,37.233960,33.376878,26.338690,16.102281,93.627380


In [33]:
dmc = dmc['2015':] 
roi = roi['2015':] 
pe = pe['2015':] 
price = price['2015':] 

In [34]:
def init_capital(quantile, ratio, price): 
    day1_ratio = ratio.iloc[0].to_frame().rename(columns={'2015-01-02': 'ratio'})
    upper = day1_ratio.quantile(quantile) 
    lower = day1_ratio.quantile(1-quantile) 
    day1_price = price.iloc[0].to_frame().rename(columns={'2015-01-02': 'price'})
    day1_ratio = day1_ratio.join(day1_price)
    for i in range(len(day1_ratio)): 
        if day1_ratio['ratio'][i] > upper: 
            day1_ratio['ratio'][i] = 1
        elif day1_ratio['ratio'][i] < lower: 
            day1_ratio['ratio'][i] = -1
        else: 
            day1_ratio['ratio'][i] = 0 
    captial = sum(abs(day1_ratio['ratio'])*day1_ratio['capital']) 
    return capital 

## Conclusion

After extracting the ratios, my approach was to get a certain quantile based on the each of the financial ratios. Then, I assigned those greater than the upper quantile to buy and those less the lower quantile to short. Since I do not have sufficient time to finish the strategy and do not want to submit this assignment so late, I cannot make any justification on the profit and loss at the end. 

However, there is one thing I could mention, which is the ticker sorting part. Since I only selected ~200 tickers, this seems to give a much more volatile result than a universe with 400 tickers or even more. This is obvious as more tickers will provide a more diversified portfolio with the same quantile range. Therefore, I could make my filtration process more precise to generate as many tickers as possible under the homework instruction. 

However, there is one drawback to have more ticker. That is higher amount of transaction fee and tax than holding less stocks in each portfolio. If we can find out an algorithm to balance to duration of updating the portfolio and holding the best quantiles, we might be able to limit the transaction fee and hold the best performed portfolio in real life. 